In [1]:
import torch
import numpy as np
from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN
from deeprobust.graph.global_attack import Metattack
import scipy.sparse as sp

In [2]:
# Load data
data = Dataset(root='/tmp/', name='cora', setting='nettack')
adj, features, labels = data.adj, data.features, data.labels
idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
idx_unlabeled = np.union1d(idx_val, idx_test)

Loading cora dataset...
Done!
Selecting 1 largest connected components


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# Train surrogate model with hidden dim = 64 (increased from 16)
surrogate = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1, nhid=64,
                with_relu=False, device=device)
surrogate = surrogate.to(device)
surrogate.fit(features, adj, labels, idx_train)

In [5]:
# Create a clean test model to evaluate attack performance
test_model = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1, nhid=64, 
                 with_relu=True, device=device)
test_model = test_model.to(device)
test_model.fit(features, adj, labels, idx_train)

In [6]:
# Set up Metattack
model = Metattack(model=surrogate, nnodes=adj.shape[0], feature_shape=features.shape, device=device)
model = model.to(device)

# Calculate total edges in the graph
total_edges = adj.sum() // 2
print(f"Total edges in the graph: {total_edges}")

# Test for different perturbation percentages
perturbation_percentages = [0.05, 0.10, 0.15, 0.20]  # 5%, 10%, 15%, 20%

results = {}

Total edges in the graph: 5069.0


In [7]:
for percentage in perturbation_percentages:
    # Calculate number of perturbations for this percentage
    perturbations = int(percentage * total_edges)
    print(f"\nRunning attack with {percentage*100}% perturbations ({perturbations} edges)")
    
    # Run the attack
    model.attack(features, adj, labels, idx_train, idx_unlabeled, perturbations, ll_constraint=False)
    modified_adj = model.modified_adj
    
    # For evasion attack - evaluate the model on the modified graph
    # (The model was trained on clean graph but evaluated on perturbed graph)
    test_model.eval()
    output = test_model.predict(features, modified_adj)
    
    # Calculate accuracy on perturbed graph
    acc_test = test_model.test(idx_test, output, labels)
    
    # Store results
    results[percentage] = {
        'perturbations': perturbations,
        'test_accuracy': acc_test
    }
    
    print(f"Test accuracy under {percentage*100}% perturbation: {acc_test:.4f}")


Running attack with 5.0% perturbations (253 edges)


Perturbing graph:   0%|          | 0/253 [00:00<?, ?it/s]

GCN loss on unlabled data: 0.5257309079170227
GCN acc on unlabled data: 0.843540455967814
attack loss: 0.17860426008701324


Perturbing graph:   0%|          | 1/253 [00:00<03:11,  1.31it/s]

GCN loss on unlabled data: 0.5278760194778442
GCN acc on unlabled data: 0.8426464014304873
attack loss: 0.17939476668834686


...

GCN loss on unlabled data: 1.3442736864089966
GCN acc on unlabled data: 0.7443004023245419
attack loss: 0.6673052906990051


Perturbing graph: 100%|█████████▉| 252/253 [01:40<00:00,  2.49it/s]

GCN loss on unlabled data: 1.3633173704147339
GCN acc on unlabled data: 0.743406347787215
attack loss: 0.6745527982711792


Perturbing graph: 100%|██████████| 253/253 [01:41<00:00,  2.50it/s]


AttributeError: 'numpy.ndarray' object has no attribute 'is_sparse'

In [1]:
import torch
import numpy as np
from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN
from deeprobust.graph.global_attack import Metattack

In [2]:
# Load the dataset
data = Dataset(root='/tmp/', name='cora', setting='nettack')
adj, features, labels = data.adj, data.features, data.labels
idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
idx_unlabeled = np.union1d(idx_val, idx_test)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Loading cora dataset...
Selecting 1 largest connected components


In [3]:
# Train a GCN on the clean graph
model_clean = GCN(nfeat=features.shape[1],
                  nclass=labels.max().item() + 1,
                  nhid=64,
                  with_relu=False,
                  dropout=0.5,
                  device=device)
model_clean = model_clean.to(device)
model_clean.fit(features, adj, labels, idx_train)

In [4]:
# Now apply Metattack, but only to modify the graph *after* model is trained
surrogate = model_clean  # surrogate can be the same trained model

# Initialize Metattack
attack_model = Metattack(model=surrogate,
                         nnodes=adj.shape[0],
                         feature_shape=features.shape,
                         attack_structure=True,
                         attack_features=False,
                         device=device)
attack_model = attack_model.to(device)

In [5]:
# Define 5% perturbations based on number of edges
perturbations = int(0.05 * (adj.sum() // 2))


In [6]:
# Run attack to generate modified adjacency matrix
attack_model.attack(features, adj, labels, idx_train, idx_unlabeled, perturbations, ll_constraint=False)
modified_adj = attack_model.modified_adj

Perturbing graph:   0%|          | 0/253 [00:00<?, ?it/s]

GCN loss on unlabled data: 0.48713213205337524
GCN acc on unlabled data: 0.8462226195797944
attack loss: 0.17344924807548523


Perturbing graph:   0%|          | 1/253 [00:00<01:45,  2.39it/s]

GCN loss on unlabled data: 0.48939570784568787
GCN acc on unlabled data: 0.8489047831917748
attack loss: 0.17622441053390503


Perturbing graph:   1%|          | 2/253 [00:00<01:36,  2.59it/s]

...

Perturbing graph:  99%|█████████▉| 251/253 [01:34<00:00,  2.75it/s]

GCN loss on unlabled data: 1.2702404260635376
GCN acc on unlabled data: 0.7152436298614215
attack loss: 0.6396557092666626


Perturbing graph: 100%|█████████▉| 252/253 [01:34<00:00,  2.74it/s]

GCN loss on unlabled data: 1.2741100788116455
GCN acc on unlabled data: 0.7125614662494413
attack loss: 0.6430190205574036


Perturbing graph: 100%|██████████| 253/253 [01:35<00:00,  2.66it/s]


In [11]:
from deeprobust.graph.utils import normalize_adj, sparse_mx_to_torch_sparse_tensor
import scipy.sparse as sp

# If modified_adj is a PyTorch tensor, convert it back to scipy first
if isinstance(modified_adj, torch.Tensor):
    modified_adj = modified_adj.cpu().numpy()
    modified_adj = sp.csr_matrix(modified_adj)  # ensure sparse format

# Normalize and convert to torch sparse tensor
modified_adj_normalized = normalize_adj(modified_adj)
modified_adj_tensor = sparse_mx_to_torch_sparse_tensor(modified_adj_normalized).to(device)

In [13]:
# Ensure features is a PyTorch tensor on correct device
if isinstance(features, np.ndarray):
    features = torch.FloatTensor(features).to(device)
elif not isinstance(features, torch.Tensor):
    features = torch.FloatTensor(features.A).to(device)
else:
    features = features.to(device)

# Evaluate model on perturbed graph
model_clean.eval()
output = model_clean.predict(features, modified_adj_tensor)
preds = output.argmax(dim=1)



In [15]:
# Compute accuracy
# Ensure labels is a PyTorch tensor
if isinstance(labels, np.ndarray):
    labels = torch.LongTensor(labels).to(device)

# Compute accuracy
correct = preds[idx_test].eq(labels[idx_test]).sum().item()
acc_evasion = correct / len(idx_test)

print(f"Test Accuracy after evasion attack: {acc_evasion:.4f}")

Test Accuracy after evasion attack: 0.8169


In [2]:
import torch
import numpy as np
import scipy.sparse as sp

from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN
from deeprobust.graph.global_attack import Metattack
from deeprobust.graph.utils import normalize_adj, sparse_mx_to_torch_sparse_tensor

# Device setup
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 1. Load the Cora dataset
data = Dataset(root='/tmp/', name='cora', setting='nettack')
adj, features, labels = data.adj, data.features, data.labels
idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
idx_unlabeled = np.union1d(idx_val, idx_test)

# 2. Train GCN on the clean graph
model_clean = GCN(nfeat=features.shape[1],
                  nclass=labels.max().item() + 1,
                  nhid=64,
                  with_relu=False,
                  dropout=0.5,
                  device=device)
model_clean = model_clean.to(device)
model_clean.fit(features, adj, labels, idx_train)

# 3. Ensure features and labels are PyTorch tensors on GPU for inference
if isinstance(features, np.ndarray):
    features = torch.FloatTensor(features).to(device)
elif not isinstance(features, torch.Tensor):
    features = torch.FloatTensor(features.A).to(device)
else:
    features = features.to(device)

if isinstance(labels, np.ndarray):
    labels = torch.LongTensor(labels).to(device)

# 4. Evaluate accuracy on the clean graph
adj_norm = normalize_adj(adj)
adj_tensor = sparse_mx_to_torch_sparse_tensor(adj_norm).to(device)

model_clean.eval()
output_clean = model_clean.predict(features, adj_tensor)
preds_clean = output_clean.argmax(dim=1)

correct_clean = preds_clean[idx_test].eq(labels[idx_test]).sum().item()
acc_clean = correct_clean / len(idx_test)
print(f"✅ Clean Graph Test Accuracy: {acc_clean:.4f}")

# 5. Prepare for Metattack (move inputs to CPU)
features_cpu = features.cpu()
labels_cpu = labels.cpu()

# 6. Apply evasion attack (perturb structure after training)
attack_model = Metattack(model=model_clean,
                         nnodes=adj.shape[0],
                         feature_shape=features.shape,
                         attack_structure=True,
                         attack_features=False,
                         device=device)
attack_model = attack_model.to(device)

perturbations = int(0.05 * (adj.sum() // 2))  # 5% of total edges
attack_model.attack(features_cpu, adj, labels_cpu, idx_train, idx_unlabeled, perturbations, ll_constraint=False)
modified_adj = attack_model.modified_adj

# 7. Normalize and convert modified adj to PyTorch sparse tensor
if isinstance(modified_adj, torch.Tensor):
    modified_adj = modified_adj.cpu().numpy()
    modified_adj = sp.csr_matrix(modified_adj)

modified_adj_norm = normalize_adj(modified_adj)
modified_adj_tensor = sparse_mx_to_torch_sparse_tensor(modified_adj_norm).to(device)

# 8. Evaluate model on the attacked graph
output_adv = model_clean.predict(features, modified_adj_tensor)
preds_adv = output_adv.argmax(dim=1)

correct_adv = preds_adv[idx_test].eq(labels[idx_test]).sum().item()
acc_evasion = correct_adv / len(idx_test)
print(f"🧨 Evasion Attack Test Accuracy: {acc_evasion:.4f}")

# 9. Compute and report accuracy drop
acc_drop = acc_clean - acc_evasion
print(f"📉 Accuracy Drop due to Evasion Attack: {acc_drop:.4f}")


Loading cora dataset...
Selecting 1 largest connected components
✅ Clean Graph Test Accuracy: 0.8260


Perturbing graph:   0%|          | 0/253 [00:00<?, ?it/s]

GCN loss on unlabled data: 0.5112600922584534
GCN acc on unlabled data: 0.8448815377738043
attack loss: 0.14672847092151642


Perturbing graph:   0%|          | 1/253 [00:00<02:22,  1.77it/s]

GCN loss on unlabled data: 0.5395905375480652
GCN acc on unlabled data: 0.8480107286544479
attack loss: 0.15828680992126465


...

GCN loss on unlabled data: 1.3060275316238403
GCN acc on unlabled data: 0.7183728207420653
attack loss: 0.5895988345146179


Perturbing graph: 100%|█████████▉| 252/253 [01:58<00:00,  2.12it/s]

GCN loss on unlabled data: 1.3051196336746216
GCN acc on unlabled data: 0.7219490388913724
attack loss: 0.5895567536354065


Perturbing graph: 100%|██████████| 253/253 [01:59<00:00,  2.12it/s]

🧨 Evasion Attack Test Accuracy: 0.8244
📉 Accuracy Drop due to Evasion Attack: 0.0015


In [2]:
import torch
import numpy as np
import scipy.sparse as sp

from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN
from deeprobust.graph.global_attack import Metattack
from deeprobust.graph.utils import normalize_adj, sparse_mx_to_torch_sparse_tensor

# Device setup
device = torch.device("cpu")

# 1. Load the Cora dataset
data = Dataset(root='/tmp/', name='cora', setting='nettack')
adj, features, labels = data.adj, data.features, data.labels
idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
idx_unlabeled = np.union1d(idx_val, idx_test)

# 2. Train GCN on the clean graph
model_clean = GCN(nfeat=features.shape[1],
                  nclass=labels.max().item() + 1,
                  nhid=64,
                  with_relu=False,
                  dropout=0.5,
                  device=device)
model_clean = model_clean.to(device)
model_clean.fit(features, adj, labels, idx_train)

# 3. Ensure features and labels are PyTorch tensors on GPU for inference
if isinstance(features, np.ndarray):
    features = torch.FloatTensor(features).to(device)
elif not isinstance(features, torch.Tensor):
    features = torch.FloatTensor(features.A).to(device)
else:
    features = features.to(device)

if isinstance(labels, np.ndarray):
    labels = torch.LongTensor(labels).to(device)

# 4. Evaluate accuracy on the clean graph
adj_norm = normalize_adj(adj)
adj_tensor = sparse_mx_to_torch_sparse_tensor(adj_norm).to(device)

model_clean.eval()
output_clean = model_clean.predict(features, adj_tensor)
preds_clean = output_clean.argmax(dim=1)

correct_clean = preds_clean[idx_test].eq(labels[idx_test]).sum().item()
acc_clean = correct_clean / len(idx_test)
print(f"✅ Clean Graph Test Accuracy: {acc_clean:.4f}")

# 5. Prepare for Metattack (move inputs to CPU)
features_cpu = features.cpu()
labels_cpu = labels.cpu()

# 6. Apply evasion attack (perturb structure after training)
attack_model = Metattack(model=model_clean,
                         nnodes=adj.shape[0],
                         feature_shape=features.shape,
                         attack_structure=True,
                         attack_features=False,
                         device=device)
attack_model = attack_model.to(device)

perturbations = int(0.10 * (adj.sum() // 2))  # 5% of total edges
attack_model.attack(features_cpu, adj, labels_cpu, idx_train, idx_unlabeled, perturbations, ll_constraint=False)
modified_adj = attack_model.modified_adj

# 7. Normalize and convert modified adj to PyTorch sparse tensor
if isinstance(modified_adj, torch.Tensor):
    modified_adj = modified_adj.cpu().numpy()
    modified_adj = sp.csr_matrix(modified_adj)

modified_adj_norm = normalize_adj(modified_adj)
modified_adj_tensor = sparse_mx_to_torch_sparse_tensor(modified_adj_norm).to(device)

# 8. Evaluate model on the attacked graph
output_adv = model_clean.predict(features, modified_adj_tensor)
preds_adv = output_adv.argmax(dim=1)

correct_adv = preds_adv[idx_test].eq(labels[idx_test]).sum().item()
acc_evasion = correct_adv / len(idx_test)
print(f"🧨 Evasion Attack Test Accuracy: {acc_evasion:.4f}")

# 9. Compute and report accuracy drop
acc_drop = acc_clean - acc_evasion
print(f"📉 Accuracy Drop due to Evasion Attack: {acc_drop:.4f}")


Loading cora dataset...
Selecting 1 largest connected components
✅ Clean Graph Test Accuracy: 0.8169


Perturbing graph:   0%|          | 0/506 [00:00<?, ?it/s]

GCN loss on unlabled data: 0.5107272863388062
GCN acc on unlabled data: 0.8466696468484578
attack loss: 0.17016395926475525


Perturbing graph:   0%|          | 1/506 [00:03<29:24,  3.49s/it]

GCN loss on unlabled data: 0.5183678865432739
GCN acc on unlabled data: 0.8480107286544479
attack loss: 0.16849106550216675


...

GCN loss on unlabled data: 2.907719135284424
GCN acc on unlabled data: 0.5686186857398301
attack loss: 1.422203540802002


Perturbing graph: 100%|█████████▉| 505/506 [21:17<00:02,  2.49s/it]

GCN loss on unlabled data: 3.0190513134002686
GCN acc on unlabled data: 0.5641484130531963
attack loss: 1.468617558479309


Perturbing graph: 100%|██████████| 506/506 [21:19<00:00,  2.53s/it]

🧨 Evasion Attack Test Accuracy: 0.8028
📉 Accuracy Drop due to Evasion Attack: 0.0141


In [ ]:
import torch
import numpy as np
import scipy.sparse as sp
import gc

from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN
from deeprobust.graph.global_attack import Metattack
from deeprobust.graph.utils import normalize_adj, sparse_mx_to_torch_sparse_tensor

# Device setup
device = torch.device("cpu")

# 1. Load the Cora dataset
data = Dataset(root='/tmp/', name='cora', setting='nettack')
adj, features, labels = data.adj, data.features, data.labels
idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
idx_unlabeled = np.union1d(idx_val, idx_test)

# 2. Train GCN on the clean graph
model_clean = GCN(nfeat=features.shape[1],
                  nclass=labels.max().item() + 1,
                  nhid=64,
                  with_relu=False,
                  dropout=0.5,
                  device=device)
model_clean = model_clean.to(device)
model_clean.fit(features, adj, labels, idx_train)

# 3. Ensure features and labels are PyTorch tensors on GPU for inference
if isinstance(features, np.ndarray):
    features = torch.FloatTensor(features).to(device)
elif not isinstance(features, torch.Tensor):
    features = torch.FloatTensor(features.A).to(device)
else:
    features = features.to(device)

if isinstance(labels, np.ndarray):
    labels = torch.LongTensor(labels).to(device)

# 4. Evaluate accuracy on the clean graph
adj_norm = normalize_adj(adj)
adj_tensor = sparse_mx_to_torch_sparse_tensor(adj_norm).to(device)

model_clean.eval()
output_clean = model_clean.predict(features, adj_tensor)
preds_clean = output_clean.argmax(dim=1)

correct_clean = preds_clean[idx_test].eq(labels[idx_test]).sum().item()
acc_clean = correct_clean / len(idx_test)
print(f"✅ Clean Graph Test Accuracy: {acc_clean:.4f}")

gc.collect()

# 5. Prepare for Metattack (move inputs to CPU)
features_cpu = features.cpu()
labels_cpu = labels.cpu()

gc.collect()

# 6. Apply evasion attack (perturb structure after training)
attack_model = Metattack(model=model_clean,
                         nnodes=adj.shape[0],
                         feature_shape=features.shape,
                         attack_structure=True,
                         attack_features=False,
                         device=device)
attack_model = attack_model.to(device)

perturbations = int(0.15 * (adj.sum() // 2))  # 5% of total edges
attack_model.attack(features_cpu, adj, labels_cpu, idx_train, idx_unlabeled, perturbations, ll_constraint=False)
modified_adj = attack_model.modified_adj

gc.collect()

# 7. Normalize and convert modified adj to PyTorch sparse tensor
if isinstance(modified_adj, torch.Tensor):
    modified_adj = modified_adj.cpu().numpy()
    modified_adj = sp.csr_matrix(modified_adj)

modified_adj_norm = normalize_adj(modified_adj)
modified_adj_tensor = sparse_mx_to_torch_sparse_tensor(modified_adj_norm).to(device)

gc.collect()

# 8. Evaluate model on the attacked graph
output_adv = model_clean.predict(features, modified_adj_tensor)
preds_adv = output_adv.argmax(dim=1)

correct_adv = preds_adv[idx_test].eq(labels[idx_test]).sum().item()
acc_evasion = correct_adv / len(idx_test)
print(f"🧨 Evasion Attack Test Accuracy: {acc_evasion:.4f}")

# 9. Compute and report accuracy drop
acc_drop = acc_clean - acc_evasion
print(f"📉 Accuracy Drop due to Evasion Attack: {acc_drop:.4f}")


Loading cora dataset...
Selecting 1 largest connected components
✅ Clean Graph Test Accuracy: 0.8159


Perturbing graph:   0%|                                                                         | 0/760 [00:00<?, ?it/s]

GCN loss on unlabled data: 0.5101723670959473
GCN acc on unlabled data: 0.8453285650424676
attack loss: 0.197859525680542


Perturbing graph:   0%|                                                               | 1/760 [00:08<1:46:53,  8.45s/it]

GCN loss on unlabled data: 0.4966593086719513
GCN acc on unlabled data: 0.8493518104604381
attack loss: 0.1960643082857132


...

GCN loss on unlabled data: 2.167794704437256
GCN acc on unlabled data: 0.62136790344211
attack loss: 1.1851763725280762


Perturbing graph:  77%|██████████████████████████████████████████████▊              | 584/760 [1:18:17<22:57,  7.83s/it]

GCN loss on unlabled data: 2.1868813037872314
GCN acc on unlabled data: 0.6227089852481001
attack loss: 1.1912404298782349


Perturbing graph:  77%|██████████████████████████████████████████████▉              | 585/760 [1:18:24<22:30,  7.72s/it]

GCN loss on unlabled data: 2.146125316619873
GCN acc on unlabled data: 0.6177916852928028
attack loss: 1.166797399520874
